In [683]:
import pyld,re,json
from pyld import jsonld

basepath = 'https://wcrp-cmip.github.io/CMIP6Plus_MIP_variables/data_descriptors/'

# python3 -m http.server

basepath = 'http://localhost:8000/data_descriptors/'


default_loader = jsonld.requests_document_loader()

In [684]:

# Recursively expand and resolve @id fields containing URLs
def recursive_expand(jsonld_doc,options=None,include_ctx=True,compact=True,child=False):
    print('!!',jsonld_doc, include_ctx, compact)
    
    loader = default_loader
    # MAKE a loader with caching capabilities
    # # Expand the current document using pyld
    ldfile = loader(jsonld_doc)['document']
    
    expanded_doc = jsonld.expand(jsonld_doc,options={'defaultLoader': loader})
    
    # Recursively check for any @id fields containing URLs
    for i,item in enumerate(expanded_doc):
    
        item = recursive_resolve_ids(item,include_ctx,compact).copy()
        
        if compact:
            include_ctx = True
        if not child:
            item = jsonld.compact(item, ldfile['@context'],options={'base': basepath})
            
        # if include_ctx:
        #     item['@context'] = ldfile['@context']
        
        # manual fix for the context
        expanded_doc[i] = json.loads(re.sub(r'"context"', '"@context"', json.dumps(item)))
        print('---------------')

    return expanded_doc

# Helper function to recursively resolve @id fields in the document
def recursive_resolve_ids(data,include_ctx,compact):
    if isinstance(data, dict):
        # If an @id field with a URL is found, fetch and expand the content
        # do not create an infinte loop if type exists. 
        if not '@type' in data and '@id' in data and data['@id'].startswith("http"):
            expanded_external_data = recursive_expand(data['@id'],include_ctx,compact,True)  # Recursive call to expand 
            print(expanded_external_data)
            # expanded_external_data[0].update(data)
            data = expanded_external_data[0]
            
            data['context'] = data['@context']
            # merge other data 
            
        for key, value in data.items():
                data[key] = recursive_resolve_ids(value,include_ctx,compact)

    elif isinstance(data, list):
        # Process lists recursively
        data = [recursive_resolve_ids(item,include_ctx,compact) for item in data]
        
    return data

In [685]:
url = f"{basepath}variables/acmonz.vmrox.json"

recursive_expand(url)

!! http://localhost:8000/data_descriptors/variables/acmonz.vmrox.json True True
!! http://localhost:8000/data_descriptors/tables/acmonz.json True True
---------------
[{'@context': 'http://localhost:8000/data_descriptors/tables/_context', 'id': 'acmonz', 'type': 'mip-table', 'Conventions': 'CF-1.7 CMIP-6.5', 'approx_interval': 30.0, 'checksum': '', 'cmor_version': '3.8.0', 'data_specs_version': '6.5.0.0', 'generic_levels': '', 'int_missing_value': '-999', 'missing_value': '1e20', 'product': 'model-output', 'table_date': '2023-11-16', 'table_id': 'ACmonZ'}]
!! http://localhost:8000/data_descriptors/tables/acmonz.json True True
---------------
[{'@context': 'http://localhost:8000/data_descriptors/tables/_context', 'id': 'acmonz', 'type': 'mip-table', 'Conventions': 'CF-1.7 CMIP-6.5', 'approx_interval': 30.0, 'checksum': '', 'cmor_version': '3.8.0', 'data_specs_version': '6.5.0.0', 'generic_levels': '', 'int_missing_value': '-999', 'missing_value': '1e20', 'product': 'model-output', 'tabl

[{'@context': 'http://localhost:8000/data_descriptors/variables/_context',
  'id': 'acmonz.vmrox.json',
  'type': 'mip-variable',
  'cell_measures': '',
  'cell_methods': 'longitude: mean time: mean',
  'comment': 'Mole Fraction of Ox',
  'dimensions': ['latitude', 'plev39', 'time'],
  'dtype': 'real',
  'frequency': 'mon',
  'long_name': 'Mole Fraction of Odd Oxygen (O, O3 and O1D)',
  'mip-tables': [{'id': 'acmonz',
    'type': 'mip-table',
    'Conventions': 'CF-1.7 CMIP-6.5',
    'approx_interval': 30.0,
    'checksum': '',
    'cmor_version': '3.8.0',
    '@context': 'http://localhost:8000/data_descriptors/tables/_context',
    'data_specs_version': '6.5.0.0',
    'generic_levels': '',
    'int_missing_value': '-999',
    'missing_value': '1e20',
    'product': 'model-output',
    'table_date': '2023-11-16',
    'table_id': 'ACmonZ'},
   {'id': 'acmonz',
    'type': 'mip-table',
    'Conventions': 'CF-1.7 CMIP-6.5',
    'approx_interval': 30.0,
    'checksum': '',
    'cmor_versio

In [253]:
# import requests

# # Custom recursive document loader
# def recursive_document_loader(url):
#     def loader(url):
#         # Fetch the primary document
#         response = requests.get(url)
#         if response.status_code != 200:
#             raise Exception(f"Failed to load document: {url}")
#         doc = response.json()

#         # Recursively fetch interdependent documents
#         for key, value in doc.items():
#             if isinstance(value, dict) and "id" in value and value["id"].startswith("http"):
#                 # Fetch the referenced document
#                 ref_doc = recursive_document_loader(value["id"])
#                 # Embed the referenced document into the current document
#                 doc[key] = ref_doc

#         return {
#             'contextUrl': None,
#             'documentUrl': url,
#             'document': doc
#         }
    
#     return loader(url)

# # Set the recursive document loader as the document loader for pyld
# def custom_document_loader(_,url):
#     print(_,url)
#     return recursive_document_loader(url)


# # options={'documentLoader': custom_document_loader}

In [290]:
from pprint import pprint
def myloader(*args, **kwargs):
    requests_loader = pyld.documentloader.requests.requests_document_loader(*args, **kwargs)
    
    def loader(url, options={}):
        print(url,options)
  
        rtn = requests_loader(url, options)
        
        # if '_context' not in url:
        #     pprint(rtn)
        
        # if aslist:
        
        #     rtn = [rtn]
        
        # pprint(rtn)
        
        return rtn
    
    return loader

# jsonld.set_document_loader(myloader())



# options={'documentLoader': loader}

In [678]:

# loader = pyld.jsonld.requests_document_loader()

frame = {
    
    "@context": "http://localhost:8000/data_descriptors/variables/_context",
    
    # "mip-tables": {"@context": "http://localhost:8000/data_descriptors/tables/_context"},
    "frequency": {},
    "test":{"@embed": "@always"},
    "@explicit": False,  
    "@embed": "@always"
}


# need to specify vicab as ""

jsonld.frame(recursive_expand(f"{basepath}variables/acmonz.vmrox.json"), frame, options={"extractAllScripts": True,"embed": "@always","explicit": False,'documentLoader':myloader()})




!! http://localhost:8000/data_descriptors/variables/acmonz.vmrox.json True True
!! http://localhost:8000/data_descriptors/tables/acmonz.json True True
---------------
[{'@context': 'http://localhost:8000/data_descriptors/tables/_context', 'id': 'acmonz', 'type': 'mip-table', 'Conventions': 'CF-1.7 CMIP-6.5', 'approx_interval': 30.0, 'checksum': '', 'cmor_version': '3.8.0', 'data_specs_version': '6.5.0.0', 'generic_levels': '', 'int_missing_value': '-999', 'missing_value': '1e20', 'product': 'model-output', 'table_date': '2023-11-16', 'table_id': 'ACmonZ'}]
!! http://localhost:8000/data_descriptors/tables/acmonz.json True True
---------------
[{'@context': 'http://localhost:8000/data_descriptors/tables/_context', 'id': 'acmonz', 'type': 'mip-table', 'Conventions': 'CF-1.7 CMIP-6.5', 'approx_interval': 30.0, 'checksum': '', 'cmor_version': '3.8.0', 'data_specs_version': '6.5.0.0', 'generic_levels': '', 'int_missing_value': '-999', 'missing_value': '1e20', 'product': 'model-output', 'tabl

{'@context': 'http://localhost:8000/data_descriptors/variables/_context',
 'id': 'acmonz.vmrox.json',
 'type': 'mip-variable',
 'cell_measures': '',
 'cell_methods': 'longitude: mean time: mean',
 'comment': 'Mole Fraction of Ox',
 'dimensions': ['latitude', 'plev39', 'time'],
 'dtype': 'real',
 'frequency': 'mon',
 'long_name': 'Mole Fraction of Odd Oxygen (O, O3 and O1D)',
 'mip-tables': {'id': 'acmonz',
  'type': 'mip-table',
  'Conventions': 'CF-1.7 CMIP-6.5',
  'approx_interval': 30.0,
  'checksum': '',
  'cmor_version': '3.8.0',
  'context': 'http://localhost:8000/data_descriptors/tables/_context',
  'data_specs_version': '6.5.0.0',
  'generic_levels': '',
  'int_missing_value': '-999',
  'missing_value': '1e20',
  'product': 'model-output',
  'table_date': '2023-11-16',
  'table_id': 'ACmonZ'},
 'modeling_realm': 'atmosChem',
 'ok_max_mean_abs': '',
 'ok_min_mean_abs': '',
 'out_name': 'vmrox',
 'positive': '',
 'standard_name': 'mole_fraction_of_ox_in_air',
 'test': None,
 'the

In [630]:
url

'http://localhost:8000/data_descriptors/variables/acmonz.vmrox.json'

In [533]:
help(jsonld.compact)

Help on function compact in module pyld.jsonld:

compact(input_, ctx, options=None)
    Performs JSON-LD compaction.
    
    :param input_: the JSON-LD input to compact.
    :param ctx: the JSON-LD context to compact with.
    :param [options]: the options to use.
      [base] the base IRI to use.
      [compactArrays] True to compact arrays to single values when
        appropriate, False not to (default: True).
      [graph] True to always output a top-level graph (default: False).
      [expandContext] a context to expand with.
      [extractAllScripts] True to extract all JSON-LD script elements
        from HTML, False to extract just the first
        (default: False).
      [processingMode] Either 'json-ld-1.0' or 'json-ld-1.1',
        defaults to 'json-ld-1.1'.
      [documentLoader(url, options)] the document loader
        (default: _default_document_loader).
    
    :return: the compacted JSON-LD output.



In [609]:
jsonld.expand(url) 

[{'http://localhost:8000/data_descriptors/variables/cell_measures': [{'@value': ''}],
  'http://localhost:8000/data_descriptors/variables/cell_methods': [{'@value': 'longitude: mean time: mean'}],
  'http://localhost:8000/data_descriptors/variables/comment': [{'@value': 'Mole Fraction of Ox'}],
  'http://localhost:8000/data_descriptors/variables/dimensions': [{'@value': 'latitude'},
   {'@value': 'plev39'},
   {'@value': 'time'}],
  'http://localhost:8000/data_descriptors/variables/dtype': [{'@value': 'real'}],
  'http://localhost:8000/data_descriptors/variables/frequency': [{'@value': 'mon'}],
  '@id': 'http://localhost:8000/data_descriptors/variables/acmonz.vmrox.json',
  'http://localhost:8000/data_descriptors/variables/long_name': [{'@value': 'Mole Fraction of Odd Oxygen (O, O3 and O1D)'}],
  'http://localhost:8000/data_descriptors/variables/mip-tables': [{'@id': 'http://localhost:8000/data_descriptors/tables/acmonz.json',
    'http://localhost:8000/data_descriptors/variables/mip-e

In [86]:
# import requests
# a = requests.get(f"{basepath}variables/acmonz.vmrox.json").json()
# a, requests.get(a['@context']).json()

In [694]:
import json
data = json.load(open('../data_descriptors/variables/graph.jsonld')) + json.load(open('../data_descriptors/tables/graph.jsonld'))

In [695]:
jsonld.frame(data,{'@type':"http://localhost:8000/data_descriptors/variables/mip-variable",'@context':data[0]['@context']})

{'@context': 'http://localhost:8000/data_descriptors/variables/_context',
 '@graph': [{'id': 'acmon.flashrate.json',
   'type': 'mip-variable',
   'cell_measures': 'area: areacella',
   'cell_methods': 'area: time: mean',
   'comment': "proposed name: lightning_flash_rate (units to be interpreted as 'counts km-2 s-1)",
   'dimensions': ['longitude', 'latitude', 'time'],
   'dtype': 'real',
   'frequency': 'mon',
   'long_name': 'Lightning Flash Rate',
   'mip-tables': {'id': 'acmon.json', 'mip-era': ['cmip6', 'cmip6plus']},
   'modeling_realm': 'atmosChem',
   'ok_max_mean_abs': '',
   'ok_min_mean_abs': '',
   'out_name': 'flashrate',
   'positive': '',
   'standard_name': 'frequency_of_lightning_flashes_per_unit_area',
   'themes': 'TBC',
   'units': 'km-2 s-1',
   'valid_max': '',
   'valid_min': ''},
  {'id': 'acmonz.oxloss.json',
   'type': 'mip-variable',
   'cell_measures': '',
   'cell_methods': 'longitude: mean time: mean',
   'comment': 'total chemical loss rate for o+o1d+o3'

In [697]:
jsonld.expand(data)[0]

{'http://localhost:8000/data_descriptors/variables/cell_measures': [{'@value': 'area: areacella'}],
 'http://localhost:8000/data_descriptors/variables/cell_methods': [{'@value': 'area: time: mean'}],
 'http://localhost:8000/data_descriptors/variables/comment': [{'@value': "proposed name: lightning_flash_rate (units to be interpreted as 'counts km-2 s-1)"}],
 'http://localhost:8000/data_descriptors/variables/dimensions': [{'@value': 'longitude'},
  {'@value': 'latitude'},
  {'@value': 'time'}],
 'http://localhost:8000/data_descriptors/variables/dtype': [{'@value': 'real'}],
 'http://localhost:8000/data_descriptors/variables/frequency': [{'@value': 'mon'}],
 '@id': 'acmon.flashrate.json',
 'http://localhost:8000/data_descriptors/variables/long_name': [{'@value': 'Lightning Flash Rate'}],
 'http://localhost:8000/data_descriptors/variables/mip-tables': [{'@id': 'acmon.json',
   'http://localhost:8000/data_descriptors/tables/mip-era': [{'@value': 'cmip6'}]},
  {'@id': 'acmon.json',
   'http

In [701]:
for i in data:
    print(i['type'])

mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable
mip-variable

In [756]:
tablectx = data[-1]['@context']
varctx = data[-1]['@context']

frame= {
    '@type':"mip-table",
    '@context': {
        **default_loader(tablectx)['document']['@context'],
                    "mip-variable": {
                        "@reverse": "mip-tables",
                        "@context":default_loader(varctx)['document']['@context'],
                    },
                    "variables": { "@reverse": "http://localhost:8000/data_descriptors/variables/mip-tables" },
                    
                    "mip-tables": {
    "@id": "http://localhost:8000/data_descriptors/variables/mip-tables",
        "@container": "@set"
    },
    "references": {
        "@reverse": "http://localhost:8000/data_descriptors/variables/mip-tables"
    },
    
    },
    "variables": {},
    "mip-references": {},
    
  
    # "@explicit": True,
    "@default": 'fsdafdsa'
}

pprint(frame)
jsonld.frame(data,frame)['@graph'][0]

{'@context': {'@base': 'http://localhost:8000/data_descriptors/tables/',
              '@vocab': 'http://localhost:8000/data_descriptors/tables/',
              'id': '@id',
              'mip-tables': {'@container': '@set',
                             '@id': 'http://localhost:8000/data_descriptors/variables/mip-tables'},
              'mip-variable': {'@context': {'@base': 'http://localhost:8000/data_descriptors/tables/',
                                            '@vocab': 'http://localhost:8000/data_descriptors/tables/',
                                            'id': '@id',
                                            'type': '@type'},
                               '@reverse': 'mip-tables'},
              'references': {'@reverse': 'mip-tables'},
              'type': '@type',
              'variables': {'@reverse': 'http://localhost:8000/data_descriptors/variables/mip-tables'}},
 '@default': 'fsdafdsa',
 '@type': 'mip-table',
 'mip-references': {},
 'variables': {}}


{'id': 'acmon',
 'type': 'mip-table',
 'Conventions': 'CF-1.7 CMIP-6.5',
 'approx_interval': 30.0,
 'checksum': '',
 'cmor_version': '3.8.0',
 'data_specs_version': '6.5.0.0',
 'generic_levels': '',
 'int_missing_value': '-999',
 'mip-references': None,
 'missing_value': '1e20',
 'product': 'model-output',
 'table_date': '2023-11-16',
 'table_id': 'ACmon'}

In [704]:
# jsonld.expand(data[0])